In [1]:
# Packages
import pandas as pd
import numpy as np
from sklearn.pipeline        import Pipeline
from sklearn.compose         import ColumnTransformer
from sklearn.preprocessing   import OneHotEncoder, FunctionTransformer
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold
from sklearn.metrics         import make_scorer
from sklearn.base            import clone
from xgboost                 import XGBRegressor
import joblib, mlflow
import mlflow.sklearn
import optuna
import warnings
from optuna.integration import OptunaSearchCV, MLflowCallback
from optuna.distributions import FloatDistribution, IntDistribution
from catboost import CatBoostRegressor, Pool

# Custom Metric for Training Feedback
def rmsle_xgb(preds, dtrain):
    y_true = dtrain.get_label()
    preds = np.maximum(preds, 0)
    rmsle = np.sqrt(np.mean((np.log1p(preds) - np.log1p(y_true)) ** 2))
    return 'rmsle', rmsle

# Custom Metric for GridSearch (wrapped in make_scorer)
def rmsle_sklearn(y_true, y_pred):
    y_pred = np.maximum(y_pred, 0)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true)) ** 2))

rmsle_scorer = make_scorer(rmsle_sklearn, greater_is_better=False)
# Data
df = pd.read_csv('playground-series-s5e5/train.csv')
y = df['Calories']
X = df.drop(columns=(['Calories', 'id']))

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=X['Sex']
)

# Custom Feature Engineering
def add_bmi_intensity(X_df: pd.DataFrame) -> pd.DataFrame:
    """Adds BMI and HeartRatexDuration features"""
    X = X_df.copy()
    X['BMI'] = (X['Weight'] / (X['Height'] / 100) ** 2).round(2)
    X['Timed_Intensity'] = X['Duration'] * X['Heart_Rate']
    return X

feat_eng = FunctionTransformer(add_bmi_intensity, validate=False)
# ──────────────────────────────────────────────────────────────────────
# MODEL & PIPELINE
# ──────────────────────────────────────────────────────────────────────
cat_model = CatBoostRegressor(
        loss_function="RMSE",
        random_seed=42,
        verbose=False,
        cat_features=["Sex"]
)

pipe = Pipeline([
    ("feat_eng", feat_eng),
    ("model", cat_model)
])
# ──────────────────────────────────────────────────────────────────────
# GRID  (prefix params with model__)
# ──────────────────────────────────────────────────────────────────────
def cat_objective(trial):
    params = {
        "model__depth":               trial.suggest_int("model__depth", 5, 10),
        "model__iterations":          trial.suggest_int("model__iterations", 600, 1600),
        "model__learning_rate":       trial.suggest_float("model__learning_rate", 0.01, 0.2, log=True),
        "model__l2_leaf_reg":         trial.suggest_float("model__l2_leaf_reg", 1, 10, log=True),
        "model__bagging_temperature": trial.suggest_float("model__bagging_temperature", 0, 1),
    }

    score = cross_val_score(
                clone(pipe).set_params(**params),
                X_train, y_train,
                scoring=rmsle_scorer,
                cv=5,
                n_jobs=-1
            ).mean()
    return score

In [2]:
# ------------------------------------------------------------------
# MLFLOW + OPTUNA LOOP  (same structure)
# ------------------------------------------------------------------
mlflow.set_tracking_uri("http://127.0.0.1:5000")


mlflow.set_experiment("Calories-CB-Optuna-V1")
mlflow.sklearn.autolog(log_models=False)

study_cb = optuna.create_study(direction="maximize", study_name="cat_rmsle_1")

with mlflow.start_run(run_name="optuna_parent_cb"):
    mlflow_cb = MLflowCallback(
        tracking_uri=mlflow.get_tracking_uri(),
        metric_name="neg_rmsle_cv",
        mlflow_kwargs={"nested": True}
    )

    study_cb.optimize(cat_objective, n_trials=50, callbacks=[mlflow_cb], show_progress_bar=True)

    mlflow.log_params(study_cb.best_trial.params)
    mlflow.log_metric("best_neg_rmsle_cv", study_cb.best_value)

[I 2025-05-19 21:35:06,179] A new study created in memory with name: cat_rmsle_1
C:\Users\afise\AppData\Local\Temp\ipykernel_16528\1734340983.py:13: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflow_cb = MLflowCallback(


  0%|          | 0/50 [00:00<?, ?it/s]

2025/05/19 21:36:40 INFO mlflow.tracking.fluent: Experiment with name 'cat_rmsle_1' does not exist. Creating a new experiment.
2025/05/19 21:36:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run 0 at: http://127.0.0.1:5000/#/experiments/10/runs/3b9dd71c8d3d4066992da5004a7080b0.
2025/05/19 21:36:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/10.


[I 2025-05-19 21:36:40,275] Trial 0 finished with value: -0.06290707511070118 and parameters: {'model__depth': 6, 'model__iterations': 1517, 'model__learning_rate': 0.019863489283838083, 'model__l2_leaf_reg': 1.642491420314437, 'model__bagging_temperature': 0.8547463660763194}. Best is trial 0 with value: -0.06290707511070118.


2025/05/19 21:37:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run 1 at: http://127.0.0.1:5000/#/experiments/10/runs/a746aba15c3347eab697f63c5f08989d.
2025/05/19 21:37:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/10.


...

[I 2025-05-19 23:07:50,178] Trial 47 finished with value: -0.060634490523562956 and parameters: {'model__depth': 9, 'model__iterations': 1376, 'model__learning_rate': 0.05974456432484234, 'model__l2_leaf_reg': 1.8515042198468634, 'model__bagging_temperature': 0.6077049142885751}. Best is trial 32 with value: -0.06045310480308199.


2025/05/19 23:09:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run 48 at: http://127.0.0.1:5000/#/experiments/10/runs/eb21c9d7fa3b4206bf93f360602408b6.
2025/05/19 23:09:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/10.


[I 2025-05-19 23:09:57,138] Trial 48 finished with value: -0.06064790072230131 and parameters: {'model__depth': 10, 'model__iterations': 1046, 'model__learning_rate': 0.07211544593055708, 'model__l2_leaf_reg': 3.276283133060642, 'model__bagging_temperature': 0.34485960990873066}. Best is trial 32 with value: -0.06045310480308199.


2025/05/19 23:11:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run 49 at: http://127.0.0.1:5000/#/experiments/10/runs/cff4d58304154bebb8ee7dfc1fef034f.
2025/05/19 23:11:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/10.
2025/05/19 23:11:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run optuna_parent_cb at: http://127.0.0.1:5000/#/experiments/9/runs/8023985e9bfc429886d770314eae140f.
2025/05/19 23:11:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/9.


[I 2025-05-19 23:11:21,113] Trial 49 finished with value: -0.06157712099719872 and parameters: {'model__depth': 7, 'model__iterations': 1249, 'model__learning_rate': 0.03743115687159948, 'model__l2_leaf_reg': 2.1177365382088715, 'model__bagging_temperature': 0.08413587583190646}. Best is trial 32 with value: -0.06045310480308199.
